In [142]:
import pandas as pd
import numpy as np
import imaplib
import email
import quopri
import re
import html
from bs4 import BeautifulSoup
import requests

In [143]:
username = 'prospectstudent@colby.edu'
password = 'Student.2017'

# username = 'haohaom2@gmail.com'
# password = '62947394'

### Accessing the email INBOX

In [426]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(username, password)
mail.list()
# Out: list of "folders" aka labels in gmail.
# mail.select("inbox") # connect to inbox.
mail.select('"Google Alerts/Completed"')

('OK', [b'893'])

In [539]:
result, data = mail.uid('search', None, "ALL") # search and return uids instead

first_email_uid = data[0].split()[0]
latest_email_uid = data[0].split()[-1]

test_email_uid = data[0].split()[-22]

ids = data[0].split()

In [540]:
# print('first email uid ', first_email_id)
# print('latest_email_uid ', latest_email_uid)

### Reading the email message

In [541]:
result, data = mail.uid('fetch', test_email_uid, '(RFC822)')
raw_email = str(data[0][1], 'utf-8')
email_message = email.message_from_string(raw_email)
print(email_message['subject'])
subject = email_message['subject']

Google Alert - "Phillip Frost"


In [542]:
def get_text_from_email(email_msg):
    if email_msg.is_multipart():
        s = ''
        for payload in email_msg.get_payload():
            s += payload.get_payload()
    else:
        s = b.get_payload()
    
    return s

In [543]:
# This cleans html stuff so I can extract the xml code
text = get_text_from_email(email_message).replace('\r\n', '').replace('=', '')

In [544]:
# extracts xml files from the raw string text
def extract_xml(raw_text):
    raw_text = raw_text.replace('\r\n', '').replace('=', '')
    m = re.findall('alerts/feeds\S+', raw_text)
    
    links = [('https://www.google.com/' + x).replace('\"', '') for x in m]
    return links

### Using Regex to get the links

In [545]:
# Passes in an xml link
# returns a list of links used in the website
def get_url_from_xml(xml_link):
    r  = requests.get(xml_link)
    soup = BeautifulSoup(r.content, 'html.parser')   # creates a BS4 from given xml link
    web_lists = soup.find_all('link')[1:]            # finds all the BS4 links
    
    urls = [re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]
    
    if not urls:
        return re.findall('(?<=url3D)(.*?)(?=\\\\u0026)', text)
    
    return urls

In [598]:
urls = get_url_from_xml(extract_xml(text)[0])
urls

['http://www.miamitodaynews.com/event/outsider-artists-from-havana/',
 'https://www.gurufocus.com/news/679424/2-stocks-move-wednesday',
 'https://investingnews.com/daily/life-science-investing/genetics-investing/opko-health-reports-first-quarter-2018-financial-results/']

In [621]:
url = get_url_from_xml(extract_xml(text)[0])[0]
url

'https://www.gurufocus.com/news/679424/2-stocks-move-wednesday'

In [622]:
# re.findall('(?<=url3D)(.*?)(?=\\\\u0026)', text)

### Scrapes each link and creates a keyward card

In [623]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [624]:
for paragraph in soup.find_all('p'):
    print(paragraph.text)



This page was added to your Bookmark.

Members Only. Please Sign Up or Log In first.

Bookmark of this page has been deleted.

Shares of OPKO Health Inc. (NASDAQ:OPK) jumped on Wednesday after the company reported a loss of 8 cents in the first quarter on revenue of $254.9 million, down 4.3% year over year. The company managed to beat earnings estimates by 5 cents and revenue expectations by $18 million.
“We are pleased to report steady sequential-quarter growth for RAYALDEE and 4Kscore, as well as improvement in our lab business from Q4 of last year,” Chairman and CEO Phillip Frost said. “The improved first quarter results were in line with our expectations, starting 2018 positively coming off challenging fourth quarter 2017 results. These factors, taken together, reflect progress across the breadth of our commercial and laboratory services.”

On the other hand, MaxLinear Inc. (NYSE:MXL) fell after reporting financial results for the first quarter. Earnings per share of 37 cents bea

In [625]:
[paragraph.text.split() for paragraph in soup.find_all('p')]

[['This', 'page', 'was', 'added', 'to', 'your', 'Bookmark.'],
 ['Members', 'Only.', 'Please', 'Sign', 'Up', 'or', 'Log', 'In', 'first.'],
 ['Bookmark', 'of', 'this', 'page', 'has', 'been', 'deleted.'],
 ['Shares',
  'of',
  'OPKO',
  'Health',
  'Inc.',
  '(NASDAQ:OPK)',
  'jumped',
  'on',
  'Wednesday',
  'after',
  'the',
  'company',
  'reported',
  'a',
  'loss',
  'of',
  '8',
  'cents',
  'in',
  'the',
  'first',
  'quarter',
  'on',
  'revenue',
  'of',
  '$254.9',
  'million,',
  'down',
  '4.3%',
  'year',
  'over',
  'year.',
  'The',
  'company',
  'managed',
  'to',
  'beat',
  'earnings',
  'estimates',
  'by',
  '5',
  'cents',
  'and',
  'revenue',
  'expectations',
  'by',
  '$18',
  'million.'],
 ['“We',
  'are',
  'pleased',
  'to',
  'report',
  'steady',
  'sequential-quarter',
  'growth',
  'for',
  'RAYALDEE',
  'and',
  '4Kscore,',
  'as',
  'well',
  'as',
  'improvement',
  'in',
  'our',
  'lab',
  'business',
  'from',
  'Q4',
  'of',
  'last',
  'year,”',


### Clean text of punctuations and store them all in a list 

In [626]:
words = []
for paragraph in soup.find_all('p'):
    words += paragraph.text.split()

In [627]:
words

['This',
 'page',
 'was',
 'added',
 'to',
 'your',
 'Bookmark.',
 'Members',
 'Only.',
 'Please',
 'Sign',
 'Up',
 'or',
 'Log',
 'In',
 'first.',
 'Bookmark',
 'of',
 'this',
 'page',
 'has',
 'been',
 'deleted.',
 'Shares',
 'of',
 'OPKO',
 'Health',
 'Inc.',
 '(NASDAQ:OPK)',
 'jumped',
 'on',
 'Wednesday',
 'after',
 'the',
 'company',
 'reported',
 'a',
 'loss',
 'of',
 '8',
 'cents',
 'in',
 'the',
 'first',
 'quarter',
 'on',
 'revenue',
 'of',
 '$254.9',
 'million,',
 'down',
 '4.3%',
 'year',
 'over',
 'year.',
 'The',
 'company',
 'managed',
 'to',
 'beat',
 'earnings',
 'estimates',
 'by',
 '5',
 'cents',
 'and',
 'revenue',
 'expectations',
 'by',
 '$18',
 'million.',
 '“We',
 'are',
 'pleased',
 'to',
 'report',
 'steady',
 'sequential-quarter',
 'growth',
 'for',
 'RAYALDEE',
 'and',
 '4Kscore,',
 'as',
 'well',
 'as',
 'improvement',
 'in',
 'our',
 'lab',
 'business',
 'from',
 'Q4',
 'of',
 'last',
 'year,”',
 'Chairman',
 'and',
 'CEO',
 'Phillip',
 'Frost',
 'said.',

In [628]:
# Saves words as a list of all the words that appear
import string
words = list(map(lambda x: x.strip(string.punctuation), words))

### Creates a set of the top 100 most common words, which will be filtered out

In [629]:
common_words = pd.read_table('google-10000-english-no-swears.txt', names=['words'], nrows=100)
common_words = set(common_words['words'].values)

### Retrieve Subject information from database

In [630]:
name = re.findall('"(.*?)"', subject)[0]
fname, lname = name.split()

In [631]:
### CHANGE PATH FOR LOCATION OF THE DATASET
path = r'\Users\xfu21\Desktop\\'
file = path + 'constituents_data.csv'

df = pd.read_csv(file, index_col=0)
df.head()

,bioConsId,bioFirstName,bioLastName,bioEmplPosition,bioEmplName,bioPrefAddrCity,bioPrefAddrState,bioSpId,Bday,MidName,NickName,age
1,1002709,Christopher,Abbott,Founder,botanica,Bainbridge Island,WA,NaN,12/28/1970,C.,Chris,47.0
2,1004278,Barbara,Brackett,NaN,NaN,Asheville,NC,NaN,NaN,M.,NaN,NaN
3,1004328,Marie,Juzova,NaN,NaN,NaN,NaN,NaN,1/13/1908,NaN,NaN,110.0
4,1004427,Donald,Emack,NaN,NaN,NaN,NaN,NaN,5/9/1908,A.,NaN,109.0
5,1004496,Francis,Smith,NaN,NaN,Naples,FL,NaN,3/3/1908,B.,NaN,110.0


In [632]:
rows = df.loc[(df['bioFirstName'] == fname) & (df['bioLastName'] == lname)]
rows

,bioConsId,bioFirstName,bioLastName,bioEmplPosition,bioEmplName,bioPrefAddrCity,bioPrefAddrState,bioSpId,Bday,MidName,NickName,age
40483,1053951,Phillip,Frost,CEO and Chairman,"OPKO Health, Inc.",Miami Beach,FL,1009702.0,NaN,NaN,NaN,NaN


In [633]:
rows['bioEmplName'].fillna('', inplace=True)
rows

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,bioConsId,bioFirstName,bioLastName,bioEmplPosition,bioEmplName,bioPrefAddrCity,bioPrefAddrState,bioSpId,Bday,MidName,NickName,age
40483,1053951,Phillip,Frost,CEO and Chairman,"OPKO Health, Inc.",Miami Beach,FL,1009702.0,NaN,NaN,NaN,NaN


In [634]:
occupations = [''.join(c for c in r if c not in string.punctuation).split() for r in rows['bioEmplName']]
occupations

[['OPKO', 'Health', 'Inc']]

In [635]:
rows

,bioConsId,bioFirstName,bioLastName,bioEmplPosition,bioEmplName,bioPrefAddrCity,bioPrefAddrState,bioSpId,Bday,MidName,NickName,age
40483,1053951,Phillip,Frost,CEO and Chairman,"OPKO Health, Inc.",Miami Beach,FL,1009702.0,NaN,NaN,NaN,NaN


In [636]:
occupation_score_arr = np.zeros(len(rows))
for i, occ in enumerate(occupations):
    if occ:                          # this matched person has a job
        occ_no_common = [j for j in occ if j not in common_words]       # strips the top 100 common words
        vec = CountVectorizer()
        X = vec.fit_transform(occ_no_common)
        occupation_score = vec.transform(words).toarray().sum()    # this number repesents the occupation aspect of the fit
        occupation_score_arr[i] = occupation_score

In [637]:
def get_occupation_score():
    occupation_score_arr = np.zeros(len(rows))
    for i, occ in enumerate(occupations):
        if occ:                          # this matched person has a job
            occ_no_common = [j for j in occ if j not in common_words]       # strips the top 100 common words
            vec = CountVectorizer()
            X = vec.fit_transform(occ_no_common)
            occupation_score = vec.transform(words).toarray().sum()    # this number repesents the occupation aspect of the fit
            occupation_score_arr[i] = occupation_score
    return occupation_score_arr.sum()

In [638]:
occupation_score_arr.sum()

4.0

In [639]:
vec = CountVectorizer()
keys = ['Colby']
X = vec.fit_transform(keys)

In [640]:
a = sum(vec.transform(words).toarray())

In [641]:
np.append(a, occupation_score_arr.sum())

array([0., 4.])

In [ ]:
# Make this work for multiple links and print all of them out

In [642]:
for link in urls:
    r  = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    words = []
    for paragraph in soup.find_all('p'):
        words += paragraph.text.split()
    
    vec = CountVectorizer()
    keys = ['Colby']
    X = vec.fit_transform(keys)

    a = sum(vec.transform(words).toarray())
    b = np.append(a, get_occupation_score())
    print(b)

[0. 4.]
[0. 4.]
[0. 4.]


In [118]:
vec.vocabulary_


{'colby': 0, 'devine': 1, 'john': 2}

In [117]:
vec.get_feature_names()

['colby', 'devine', 'john']

### Uses count vectorizer to get a count of key words
###### Make sure you take in account of when multiple people have the same name

#### The raw data passed through the classifier will have the format [fname, lname, Colby, occupation_score]

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [113]:
vec = CountVectorizer()
keys = [fname, lname, 'Colby']
X = vec.fit_transform(keys)

In [114]:
sum(vec.transform(words).toarray(), 1) - 1

array([ 0, 21, 21], dtype=int64)

In [27]:
occupation_vec = CountVectorizer()
occupation_array = row['bioEmplName'].str.split()
# occupation_array = [w if w not in common_words else 'a' for w in occupation_array]
for w in occupation_array:
#     if not w in common_words:
      print(w)
      print(type(w))

['Robinson,', 'Banks,', '&', 'Anderson']
<class 'list'>


In [28]:
type(occupation_array.tolist())

list

In [29]:
'but' in common_words

True

In [48]:
occupation_array

215    [Robinson,, Banks,, &, Anderson]
Name: bioEmplName, dtype: object

In [54]:
a = row['bioEmplName']
a

215    Robinson, Banks, & Anderson
Name: bioEmplName, dtype: object

In [61]:
''.join(c for c in a.values[0] if c not in string.punctuation).split()

['Robinson', 'Banks', 'Anderson']

In [40]:
'Banks' in a

False

In [41]:
set(a)

{'Robinson, Banks, & Anderson'}

In [30]:

m = re.findall('alerts/feeds\S+', text)
url = ('https://www.google.com/' + m[0]).replace('\"', '')
url

'https://www.google.com/alerts/feeds/16218955178302834825/8278055857187349425'

In [31]:
r  = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [32]:
# use &ct as a stopper

web_lists = soup.find_all('link')[1:]
for web in web_lists:
    print(web['href'])

https://www.google.com/url?rct=j&sa=t&url=http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/&ct=ga&cd=CAIyGjQxZWUzODg3MmI2ZDRmYmE6Y29tOmVuOlVT&usg=AFQjCNFJUfO5HZt9uN0OEXWnBxlF2r4aaQ


In [33]:
url = web_lists[0]['href']
url

'https://www.google.com/url?rct=j&sa=t&url=http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/&ct=ga&cd=CAIyGjQxZWUzODg3MmI2ZDRmYmE6Y29tOmVuOlVT&usg=AFQjCNFJUfO5HZt9uN0OEXWnBxlF2r4aaQ'

In [34]:
[re.findall('url=(.*?)&', url['href'])[0] for url in web_lists]

['http://www.millburysutton.com/articles/millbury-jr-sr-high-school-honor-roll-4/']

In [110]:
test_soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [111]:
test_soup.get_text()

'window.googleJavaScriptRedirect=1var n={navigateTo:function(b,a,d){if(b!=a&&b.google){if(b.google.r){b.google.r=0;b.location.href=d;a.location.replace("about:blank");}}else{a.location.replace(d);}}};n.navigateTo(window.parent,window,"http://www.nydailynews.com/new-york/woman-gave-no-hints-trouble-found-butchered-park-article-1.3955035");\n'

In [89]:
def get_first_text_block(email_message_instance):
    maintype = email_message_instance.get_content_maintype()
    if maintype == 'multipart':
        for part in email_message_instance.get_payload():
            if part.get_content_maintype() == 'text':
                return part.get_payload()
    elif maintype == 'text':
        return email_message_instance.get_payload()

In [107]:
# get_first_text_block(email_message)

#### using mail.search()

In [70]:
type, data = mail.search(None, 'ALL')
mail_ids = data[0].decode()
id_list = mail_ids.split()

In [71]:
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])

In [89]:
typ, data = mail.fetch('4', '(RFC822)' )

In [91]:
for response_part in data:
    if isinstance(response_part, tuple):
        msg = email.message_from_string(str(response_part[1]))
        email_subject = msg['subject']
        email_from = msg['from']
        print( 'From : ' + str(email_from) + '\n')
        print ('Subject : ' + str(email_subject) + '\n')

From : None

Subject : None



In [97]:
raw_email = data[0][1]
email_message = email.message_from_string(str(raw_email))
print(email_message['To'])

None


In [84]:
for response_part in data:
    if isinstance(response_part, tuple):
        print(type(response_part[1]))

<class 'bytes'>


In [65]:
import smtplib
import time
import imaplib
import email
def read_email_from_gmail(user, pw):
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(user, pw)
        mail.select('inbox')

        typ, data = mail.search(None, 'ALL')
        mail_ids = data[0]

        id_list = mail_ids.split()   
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])


        for i in range(latest_email_id,first_email_id, -1):
            typ, data = mail.fetch(str(i), '(RFC822)' )

            for response_part in data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_string(response_part[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print( 'From : ' + email_from + '\n')
                    print ('Subject : ' + email_subject + '\n')

    except Exception as e:
        print (e)

In [66]:
read_email_from_gmail(username, password)

initial_value must be str or None, not bytes
